In [21]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime, timedelta
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle


import sys, os
sys.path.append(os.path.abspath('../..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

# Machine learning libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [33]:
pair = 'DOGE_USDT'
timeframe = '4h'

exchange = 'binance'
odf = pd.read_json(f'../../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

(8833, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2019-07-05 12:00:00,1562328000000,0.004490,0.004600,0.003710,0.003738,1.314757e+09
2019-07-05 16:00:00,1562342400000,0.003728,0.003734,0.003550,0.003688,4.005844e+08
2019-07-05 20:00:00,1562356800000,0.003688,0.003874,0.003624,0.003870,2.129565e+08
2019-07-06 00:00:00,1562371200000,0.003874,0.003943,0.003704,0.003767,2.791104e+08


In [99]:
def add_indicators(wdf):
    # print(f'wdf: {wdf.shape}')
    # wdf = odf['2020-07-06':].copy()
    wdf['bod'] = wdf[['open', 'close']].min(axis=1)
    wdf['bou'] = wdf[['open', 'close']].max(axis=1)
    peak_dist = 5
    i_upeaks, _ = find_peaks(wdf.close, distance=peak_dist)
    ix_upeaks = wdf.index[i_upeaks]
    # i_dpeaks, _ = find_peaks(-wdf.close, distance=peak_dist)
    # ix_dpeaks = wdf.index[i_dpeaks]

    ##########################################

    wdf['inow'] = range(wdf.shape[0])
    wdf['lret'] = wdf.close.divide(wdf.close.shift()).apply(np.log)
    a_last_peaks = np.array([
        # wdf.index.values[[idf, ipk]]
        [idf, ipk, *wdf.index[[idf, ipk]]]
        # [(idf, ipk), tuple(wdf.index[[idf, ipk]])]
        for ipk in i_upeaks

        for idf in list(range(ipk + 1, min(wdf.shape[0] - 1, ipk + 1 + int(peak_dist/2))))
    ]).T
    # print(a_last_peaks[1])
    # if len(a_last_peaks) > 0:
    wdf.loc[a_last_peaks[2],'iupk'] = a_last_peaks[1]
    wdf.loc[a_last_peaks[2],'ixupk'] = a_last_peaks[3]

    wdf.loc[a_last_peaks[2],'n_upk'] = wdf.loc[a_last_peaks[2],'inow'] - a_last_peaks[1]

    wdf.loc[a_last_peaks[2], 'upk_close'] = wdf.loc[a_last_peaks[3]].close.values
    wdf.loc[a_last_peaks[2], 'upk_ret'] = wdf.loc[a_last_peaks[2], 'close'] / wdf.loc[a_last_peaks[2], 'upk_close']  - 1
        # wdf[['upk_ret']].describe()

    fus = [1,2,3,4,5,6]
    furet_cols = [f'furet_{fu}' for fu in fus]
    # a_last_peaks[[1,3]]

    wdf = wdf.join(pd.concat([ wdf.close.shift(-fu) / wdf.close - 1 for fu in fus ], axis=1).set_axis(furet_cols, axis=1)
                )

    lags = [2,3,4,6]
    la_cols = [f'bod_std_{la}' for la in lags]

    wdf = wdf.join(pd.concat([
        wdf.bod.rolling(la).std()
        for la in lags
        ],axis=1).set_axis(la_cols, axis=1))
    wdf.sort_index(inplace=True)

    return wdf.dropna()

# wdf = add_indicators(odf.copy())
wdf = odf[:'2022-01-01'].copy()

#########################################################################

features = ['upk_ret', 'n_upk', 'bod_std_6', 'bod_std_3', 'bod_std_2' ]
# load the model from disk
knn = pickle.load(open('./models/bottomOut-DOGE_USDT-4h--1001--knnpickle', 'rb'))
# result = loaded_model.predict(X_test) 
i = -1
pos = {}
cumpnl = 0
for index, row in wdf.iterrows():
    i += 1
    df_pre = wdf.loc[:index].tail(51).head(50)
    if df_pre.shape[0] < 50:
        # print(df_pre.shape[0])
        continue
    # print(f'dpre: {df_pre.shape}')
    df_pre = add_indicators(df_pre)

    y_hat = knn.predict(df_pre[features])
    # print(index + timedelta(hours=4*3))
    # print(index, index + timedelta(hours=6*3))
    # print('index: ', str(index), str(index + timedelta(hours=4*3)))
    pos[ str(index + timedelta(hours=4*3))] = {'y_hat': y_hat[-1], 'close': row.close}
    if str(index) in pos:
        dapos = pos[str(index)]
        pnl = log(row.close/dapos['close']) * dapos['y_hat'] - 0.002
        cumpnl += pnl
    if i % (6 * 3) == 0: 
        print(f'Day {i // 6}  {index}: pnl={round(pnl,4)}, cumpnl={round(cumpnl,4)}')


Day 9  2019-07-14 12:00:00: pnl=0.0704, cumpnl=0.1184
Day 12  2019-07-17 12:00:00: pnl=-0.0483, cumpnl=0.3852
Day 15  2019-07-20 12:00:00: pnl=-0.0151, cumpnl=0.1284
Day 18  2019-07-23 12:00:00: pnl=-0.036, cumpnl=0.0176
Day 21  2019-07-26 12:00:00: pnl=-0.0146, cumpnl=-0.1585
Day 24  2019-07-29 12:00:00: pnl=-0.0119, cumpnl=-0.0794
Day 27  2019-08-01 12:00:00: pnl=0.0049, cumpnl=-0.2257
Day 30  2019-08-04 12:00:00: pnl=-0.0119, cumpnl=-0.3134
Day 33  2019-08-07 12:00:00: pnl=0.0137, cumpnl=-0.5234
Day 36  2019-08-10 12:00:00: pnl=0.0216, cumpnl=-0.5863
Day 39  2019-08-13 12:00:00: pnl=-0.0117, cumpnl=-0.5892
Day 42  2019-08-16 16:00:00: pnl=-0.0425, cumpnl=-0.4129
Day 45  2019-08-19 16:00:00: pnl=-0.0164, cumpnl=-0.6552
Day 48  2019-08-22 16:00:00: pnl=-0.0168, cumpnl=-0.6544
Day 51  2019-08-25 16:00:00: pnl=0.0059, cumpnl=-0.6739
Day 54  2019-08-28 16:00:00: pnl=0.0275, cumpnl=-0.6286
Day 57  2019-08-31 16:00:00: pnl=-0.0051, cumpnl=-0.5544
Day 60  2019-09-03 16:00:00: pnl=-0.0165, c

KeyboardInterrupt: 

In [88]:
str(wdf.index[0])

'2019-07-05 12:00:00'

In [60]:
wdf = odf['2022-07-01':].head(50).copy()


# wdf = odf['2020-07-06':].copy()
wdf['bod'] = wdf[['open', 'close']].min(axis=1)
wdf['bou'] = wdf[['open', 'close']].max(axis=1)
peak_dist = 5
i_upeaks, _ = find_peaks(wdf.close, distance=peak_dist)
ix_upeaks = wdf.index[i_upeaks]
# i_dpeaks, _ = find_peaks(-wdf.close, distance=peak_dist)
# ix_dpeaks = wdf.index[i_dpeaks]

##########################################

wdf['inow'] = range(wdf.shape[0])
wdf['lret'] = wdf.close.divide(wdf.close.shift()).apply(np.log)
a_last_peaks = np.array([
    # wdf.index.values[[idf, ipk]]
    [idf, ipk, *wdf.index[[idf, ipk]]]
    # [(idf, ipk), tuple(wdf.index[[idf, ipk]])]
    for ipk in i_upeaks

    for idf in list(range(ipk + 1, min(wdf.shape[0] - 1, ipk + 1 + int(peak_dist/2))))
]).T
# print(a_last_peaks)
# if len(a_last_peaks) > 0:
wdf.loc[a_last_peaks[2],'iupk'] = a_last_peaks[1]
wdf.loc[a_last_peaks[2],'ixupk'] = a_last_peaks[3]

wdf.loc[a_last_peaks[2],'n_upk'] = wdf.loc[a_last_peaks[2],'inow'] - a_last_peaks[1]

wdf.loc[a_last_peaks[2], 'upk_close'] = wdf.loc[a_last_peaks[3]].close.values
wdf.loc[a_last_peaks[2], 'upk_ret'] = wdf.loc[a_last_peaks[2], 'close'] / wdf.loc[a_last_peaks[2], 'upk_close']  - 1
    # wdf[['upk_ret']].describe()
wdf


,timestamp,open,high,low,close,volume,bod,bou,inow,lret,iupk,ixupk,n_upk,upk_close,upk_ret
dtime,,,,,,,,,,,,,,,
2022-07-01 00:00:00,1656633600000,0.06642,0.06846,0.06557,0.06602,217445194.0,0.06602,0.06642,0,NaN,NaN,NaT,NaN,NaN,NaN
2022-07-01 04:00:00,1656648000000,0.06602,0.06658,0.06417,0.06626,161168564.0,0.06602,0.06626,1,0.003629,NaN,NaT,NaN,NaN,NaN
2022-07-01 08:00:00,1656662400000,0.06627,0.06686,0.06386,0.06482,147900544.0,0.06482,0.06627,2,-0.021972,1,2022-07-01 04:00:00,1,0.06626,-0.021733
2022-07-01 12:00:00,1656676800000,0.06481,0.06633,0.06435,0.06520,134092161.0,0.06481,0.06520,3,0.005845,1,2022-07-01 04:00:00,2,0.06626,-0.015998
2022-07-01 16:00:00,1656691200000,0.06518,0.06693,0.06503,0.06638,108939371.0,0.06518,0.06638,4,0.017936,NaN,NaT,NaN,NaN,NaN
2022-07-01 20:00:00,1656705600000,0.06638,0.06765,0.06578,0.06644,103204088.0,0.06638,0.06644,5,0.000903,NaN,NaT,NaN,NaN,NaN
2022-07-02 00:00:00,1656720000000,0.06645,0.06788,0.06550,0.06652,200943254.0,0.06645,0.06652,6,0.001203,NaN,NaT,NaN,NaN,NaN
2022-07-02 04:00:00,1656734400000,0.06652,0.06722,0.06612,0.06657,96203568.0,0.06652,0.06657,7,0.000751,NaN,NaT,NaN,NaN,NaN
2022-07-02 08:00:00,1656748800000,0.06657,0.06761,0.06614,0.06716,103048206.0,0.06657,0.06716,8,0.008824,NaN,NaT,NaN,NaN,NaN
